# Plotting

The next step in exploratory data analysis is plotting. Let's go through the world's fastest introduction to the core plotting library in Python, **matplotlib**.

We are going to look briefly at some of the underlying concepts in matplotlib to aid in orientation. Having a mental model of how matplotlib works will greatly aid your ability to unlock the power of matplotlib. That said, one of the best ways to make great plots and to customize matplotlib is to go to the [gallery](https://matplotlib.org/gallery.html), find a plot that you want to emulate, and read the code. Another recent but great resource is the [python graph gallery](https://python-graph-gallery.com/).

There are two interfaces available when working with matplotlib -- an interactive interface and an object-oriented interface. Both are available through the `matplotlib.pyplot` namespace.

When working interactively, matplotlib maintains a global state for your current plot.

First, let's set some Notebook display options. For more information on this, check [here](http://ipython.readthedocs.io/en/stable/interactive/plotting.html).

In [ ]:
%matplotlib inline

This is the canonical import for matplotlib.

In [ ]:
import matplotlib.pyplot as plt

Now, let's use the interactive interface.

In [ ]:
plt.plot(range(10), range(10))

plt.xlabel("X Axis")
plt.xticks([])
plt.title("The Image Title");

I don't *really* recommend using the interactive interface in practice or for anything serious. 

Alternatively, you can use the object-oriented interface. This usually entails a call to **plt.subplots**.

This call returns instances of two objects -- a Figure object and an Axes object. Understanding the [anatomy of a matplotlib plot](http://matplotlib.org/faq/usage_faq.html#parts-of-a-figure) and these two underlying concepts will help unlock the power of **matplotlib**.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

The Figure is the object that contains the Axes. To be concrete, we'll turn the figure color gray.

In [ ]:
fig.set_facecolor('gray')
fig

The Axes contain the plot.

In [ ]:
ax.scatter(
    (.25, .25, .75, .75), (.25, .75, .25, .75), 
    s=64**2,
    c=['red', 'green', 'blue', 'orange']
)

ax.set_xticks([])
ax.set_yticks([])

fig

A Figure can contain multiple Axes.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 12))

axes[0].plot(range(10), range(10))
axes[1].plot(range(10), range(10, 0, -1))

axes[0].set_title("Line 1")
axes[1].set_title("Line 2")

## High-Level Plotting

One of the reasons that this abstraction and the distinction between figures and axes can be useful is in understanding how to operate with some higher-level plotting libraries like **pandas** and **seaborn**.

Pandas itself has a plotting API that utilizes matplotlib under the hood. Let's take a look at some of this functionality using the canonical Titanic dataset with information on passenger survival in this famous shipwreck.

In [ ]:
import pandas as pd

titanic = pd.read_csv("data/titanic.csv")

As we know, bar plots can be useful for displaying and comparing measurable quantities, such as counts or volumes. Let's say we want to make a bar plot of the number of survivors by the class of their ticket. 

Pandas provides access to all of the plotting functionality through the **DataFrame.plot** attribute. The plot attribute is exposed on GroupBy objects as well. Here, we'll group by the class of the ticket and then call `...plot.bar` to get a nice bar plot.

In [ ]:
ax = (titanic.groupby('pclass')
      .survived.sum()
      .plot.bar())

Notice that pandas returned to us the matplotlib axes. We could go on and customize these axes should we so desire. Let's rotate **tick labels**, for example.

Note that each axes holds a reference to the figure, which we use to display in the notebook.

In [ ]:
for label in ax.xaxis.get_ticklabels():
    label.set_rotation(0)
    
ax.figure

We can also use this pattern to plot multiple things on an axes using a mix of pure matplotlib and higher-level plotting packages.

This plot is a little small. One trick I like to use is explicitly passing in an axes that I have already created. 

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

ax = (titanic.groupby('pclass')
      .survived.sum()
      .plot.bar(ax=ax))

Then we might want to make the tick and axes labels bigger. We could do this by munging the axes or we could take advantage of the matplotlib style context manager.

In [ ]:
font_config = {
    'font.size': 16,
}

with plt.style.context(font_config):
    fig, ax = plt.subplots(figsize=(12, 6))

    ax = (titanic.groupby('pclass')
          .survived.sum()
          .plot.bar(ax=ax))
    
    for tick in ax.xaxis.get_ticklabels():
        tick.set_rotation(0)

You can also set matplotlib style state globally using the `rc` function. See here for more information. TODO: add link.

In [ ]:
plt.rc('font', size=16)

There are a few more interesting bar plots we can make using pandas. We might want to see the number of survivors broken down with class by gender, and plot it as a horizontal bar chart.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

ax = (titanic.groupby(['sex', 'pclass'])
      .survived.sum().plot.barh(ax=ax))

We can look use a cross-tab of the those who did and did not survive by class and gender to make a stacked bar chart, which is a little more economical.

In [ ]:
death_counts = pd.crosstab([titanic.pclass, titanic.sex], titanic.survived.astype(bool))
death_counts

In [ ]:
ax = death_counts.plot.barh(stacked=True) 

We can also look at the survival *rate* of the groups, by adjusting for the number of people in each group.

In [ ]:
ax = (death_counts.div(death_counts.sum(1), axis=0)
      .plot.barh(stacked=True))

This, obviously, isn't quite what we want for the legend. We might Consider increasing the margin at the bottom and making the legend horizontal. Let's go ahead and clean up the labels too.

In [ ]:
ax = (death_counts.div(death_counts.sum(1), axis=0)
      .plot.barh(stacked=True))

ax.margins(x=0)
ax.set_ylim(-1.75, 5.5)

ax.legend(loc=(.15, .05), ncol=2)

## Health Inspection Data

Let's take a look back at the Chicago Health Inspection data.

In [ ]:
from load_data import dta

In [ ]:
dta = dta.drop(["violations"], axis='columns').join(
    dta.violations.str.split("|", expand=True)
        .unstack()
        .dropna()
        .str.strip()
        .reset_index(level=0, drop=True)
        .to_frame()
        .rename(columns={0: 'violations'}),
    how='right'
)

One thing we may wonder is if the violations are fairly evenly distributed or if some are much more common than others.

In [ ]:
violation_num = dta.violations.str.extract("(\d+)(?=\.)", expand=False).astype(int)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))

ax = violation_num.groupby(violation_num).size().plot.bar(ax=ax)

### Exercise:

See if there are any meaningful differences in the number of violations by quarter. Recall that you can use the `dt` accessor for datetime functionality on a Series.

In [ ]:
%load solutions/violation_by_quarter.py

## Plotting Geographic Data

One the most interesting features that we have here are geographical. There are a few options for plotting maps in Python (and in Notebooks).

The first thing we can try is plotting with matplotlib only. If you don't need much else, this can work really well to get a sense of how things are, if your data are sufficiently dense.

In [ ]:
dta.info()

In [ ]:
dta = dta.dropna(subset=["latitude", "longitude"])

In [ ]:
with plt.style.context({
    'axes.facecolor': 'black',
   }):
    fig, ax = plt.subplots(figsize=(16, 12))

    ax = dta.plot.scatter(x='longitude', y='latitude', color='white', s=.02, alpha=.6, ax=ax)
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticks([])
    ax.set_yticks([])

A few things to note. First, we used a context manager again to set some style that we know we wanted to adhere to. Second, we used the `DataFrame.plot.scatter` method.

Suppose we thought there may be some geographic clustering to the restaurant incident reporting.

In [ ]:
cmap = plt.cm.get_cmap('RdYlBu')


with plt.style.context({
    'axes.facecolor': 'black',
    'legend.facecolor': 'inherit',
    'legend.markerscale': 32,
    'text.color': 'white',
   }):
    fig, ax = plt.subplots(figsize=(16, 12))

    grouper = dta.groupby('facility_type')
    for i, (name, group) in enumerate(grouper):
        if len(group) < 5000:
            continue
        ax = group.plot.scatter(x='longitude', y='latitude', s=.2, color=cmap(i), alpha=.6, ax=ax, label=name)
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.legend()

Here we attempt to uncover some more detail from our map, by plotting the different types of estbalishments using different colors. We use matplotlib's **color maps** under the `plt.cmap` namespace to do this. We also add a **legend** and style it. In general, you use the `label` argument to add a name to plot element and the call `legend` on the axes to display it.

Incidentally, the matplotlib documentation has a [fantastic page](https://matplotlib.org/users/colormaps.html) about choosing color maps purposefully for different situations.

For some more fine-grained control over maps, you may want to check out packages like [cartopy](http://scitools.org.uk/cartopy/), [folium](http://python-visualization.github.io/folium/), and [bqplot](https://github.com/bloomberg/bqplot).